# Requesting External Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pprint import pprint
import requests
import numpy as np
import pandas as pd
import time
from tqdm.notebook import trange, tqdm
import pdb

In [3]:
from request_discos import *
from request_uber_ares import *

In [4]:
def getRelDataID(data):
    if type(data)==dict:
        return int(data['id'])
    elif type(data)==list:
        if len(data)==0:
            return None
        else:
            return [int(d['id']) for d in data]
    else:
        return None

In [5]:
def data2df(data_list,att_list=[]):
    ind = [int(d['id']) for d in data_list]
    attributes = [d['attributes'] for d in data_list]
    data_df = pd.DataFrame(attributes,index=ind)

    for att in att_list:
        data_df[att] = [ getRelDataID(data['relationships'][att]['data']) for data in data_list]
    data_df.sort_index(inplace=True)
    return data_df

# DiscosWeb

## Objects

In [10]:
dataRequest[0]

{'type': 'object',
 'id': '8060',
 'attributes': {'cosparId': '1975-072A',
  'name': 'COS B',
  'depth': None,
  'xSectMin': 1.539380400259,
  'diameter': 1.4,
  'span': 1.4,
  'xSectAvg': 2.08915911463721,
  'satno': 8062,
  'shape': 'Cyl',
  'objectClass': 'Payload',
  'mass': 272.55,
  'height': 1.2,
  'width': None,
  'vimpelId': None,
  'xSectMax': 2.27861625042515},
 'relationships': {'operators': {'links': {'self': '/api/objects/8060/relationships/operators',
    'related': '/api/objects/8060/operators'},
   'data': [{'type': 'organisation', 'id': '283'}]},
  'reentry': {'links': {'self': '/api/objects/8060/relationships/reentry',
    'related': '/api/objects/8060/reentry'}},
  'initialOrbits': {'links': {'self': '/api/objects/8060/relationships/initial-orbits',
    'related': '/api/objects/8060/initial-orbits'}},
  'destinationOrbits': {'links': {'self': '/api/objects/8060/relationships/destination-orbits',
    'related': '/api/objects/8060/destination-orbits'}},
  'launch': {'

In [9]:
operatorName = 'European Space Agency'
opCond = f"eq(operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(filter=filter,include='operators,states')

objectIndex = [d['id'] for d in dataRequest]
objectAttributes = [d['attributes'] for d in dataRequest]
object_df = pd.DataFrame(objectAttributes,index=objectIndex)

object_df

,cosparId,name,depth,xSectMin,diameter,span,xSectAvg,satno,shape,objectClass,mass,height,width,vimpelId,xSectMax
8060,1975-072A,COS B,NaN,1.539380,1.40,1.40,2.089159,8062,Cyl,Payload,272.55,1.20,NaN,NaN,2.278616
9929,1977-029A,ESA GEOS 1,NaN,2.010619,1.60,40.00,5.977604,9931,Cyl + 2 Ant,Payload,270.56,1.30,NaN,NaN,10.138289
10421,1977-102B,ISEE 2,NaN,1.266769,1.27,1.27,1.770484,10423,Cyl,Payload,164.52,1.14,NaN,NaN,1.923754
10635,1978-012A,IUE,NaN,1.385929,1.40,4.30,6.293122,10637,Oct Cyl + 2 Pan,Payload,457.88,4.20,NaN,NaN,9.289332
10853,1978-044A,OTS 2,NaN,3.325538,2.40,2.40,4.750615,10855,Hex Cyl,Payload,440.04,1.60,NaN,NaN,5.361194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66965,1981-057G,CAT 3 operational debris,NaN,NaN,NaN,NaN,NaN,48343,None,Payload Mission Related Object,NaN,NaN,NaN,NaN,NaN
67858,2021-073C,Sunstorm,0.1,0.010000,NaN,0.20,0.025000,49068,Box,Payload,2.00,0.20,0.1,NaN,0.030000
69581,2021-130A,James Webb Space Telescope,4.5,20.250000,NaN,21.20,66.244667,50463,Box + 1 Dish + 2 Pan,Payload,6161.00,8.00,4.5,NaN,114.911356
71827,1998-067TM,ISS operational debris (ERA MLI bundle),0.5,0.250000,NaN,0.50,0.375000,52329,Irr,Payload Mission Related Object,5.00,0.50,0.5,NaN,0.433013


## Fragmentations

In [6]:
operatorName = 'European Space Agency'
opCond = f"eq(objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/fragmentations",filter=filter,sort='epoch',include='objects')

fragIndex = [d['id'] for d in dataRequest]
fragAttributes = [d['attributes'] for d in dataRequest]
frag_df = pd.DataFrame(fragAttributes,index=fragIndex)
frag_df['ObjectID'] = [d['relationships']['objects']['data'][0]['id'] for d in dataRequest]

frag_df

,latitude,comment,epoch,longitude,altitude,eventType,ObjectID
115,None,This mission was the inaugural flight of the A...,1980-03-31,None,None,Ariane Upper Stage,11656
467,None,Provisional breakup,1984-02-27,None,None,Ariane Upper Stage,14420
353,None,Provisional breakup,1984-07-04,None,None,Ariane Upper Stage,14784
427,None,,1987-06-07,None,None,Ariane Upper Stage,15873
11,None,Above elements are initial published values fo...,1987-09-19,None,None,Ariane Upper Stage,18349
337,None,ERS-2 anomaly,2003-07-24,None,None,ERS/SPOT Class,23557
695,None,A single fragment was catalogued. The breakup ...,2007-04-15,None,None,ERS/SPOT Class,23557
748,None,"On 22 May 2007 at about 0050 UTC, the spin-sta...",2007-05-22,None,None,Small Impactor,27506
697,None,Three fragments with consecutive satellite cat...,2009-04-25,None,None,ERS/SPOT Class,23557
324,None,ERS-1 anomalous event,2009-06-03,None,None,ERS/SPOT Class,21571


## Launches

In [7]:
operatorName = 'European Space Agency'
opCond = f"eq(objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/launches",filter=filter,sort='epoch',include='objects,site')

launchIndex = [d['id'] for d in dataRequest]
launchAttributes = [d['attributes'] for d in dataRequest]
launch_df = pd.DataFrame(launchAttributes,index=launchIndex)
launch_df['SiteID'] = [d['relationships']['site']['data']['id'] for d in dataRequest]

launchObjects = []
for d in dataRequest:
    objectData = d['relationships']['objects']['data']
    launchObjects.append([obj['id'] for obj in objectData])
launch_df['ObjectIDs'] = launchObjects

launch_df.head(10)

,failure,epoch,cosparLaunchNo,flightNo,SiteID,ObjectIDs
1962,False,1975-08-09T01:48:00+00:00,1975-072,602/D113,19,"[52335, 8061, 8060]"
245,False,1977-04-20T10:15:00+00:00,1977-029,617/D130,10,"[9932, 9931, 9930, 9929]"
1603,False,1977-10-22T13:53:00+00:00,1977-102,623/D135,10,"[10423, 10422, 10421, 10420]"
1605,False,1977-11-23T01:35:00+00:00,1977-108,620/D136,10,"[48118, 48117, 13904, 10948, 10488, 10487]"
2489,False,1978-01-26T17:36:00+00:00,1978-012,628/D138,10,"[37759, 32997, 10721, 10636, 10635]"
126,False,1978-05-11T22:59:00+00:00,1978-044,627/D141,10,"[10855, 10854, 10853]"
991,False,1978-07-14T10:43:00+00:00,1978-071,631/D143,10,"[10981, 10980, 10979]"
1357,False,1979-12-24T17:14:38+00:00,1979-104,L01,23,"[66929, 40776, 40779, 40778, 40777, 40775, 407..."
361,False,1981-06-19T12:32:59+00:00,1981-057,L03,23,"[66965, 48120, 48119, 20834, 14122, 12559, 125..."
2654,False,1981-12-20T01:29:00+00:00,1981-122,L04,23,"[13008, 13007, 13022]"


## Launch Sites

In [8]:
operatorName = 'European Space Agency'
opCond = f"eq(launches.objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/launch-sites",filter=filter,include='operators')

launchsiteIndex = [d['id'] for d in dataRequest]
launchsiteAttributes = [d['attributes'] for d in dataRequest]
launchsite_df = pd.DataFrame(launchsiteAttributes,index=launchsiteIndex)

launchsite_df

,pads,latitude,constraints,longitude,altitude,azimuths,name
7,None,13.900000,None,80.400000,0.0,"[{'lower': 102.0, 'display': '[102,139]', 'low...",Sriharikota Space Center
10,[LC46],28.500000,None,-81.000000,0.0,"[{'lower': 37.0, 'display': '[37,120]', 'lower...",Kennedy Space Center (ETR)
19,"[ABRESA2, ABRESB3, BMRSA1, RW30/12PAWA, 576E, ...",34.700000,None,-120.600000,0.0,"[{'lower': 147.0, 'display': '[147,201]', 'low...",Vandenberg AFB (WTR)
21,"[LA2?, LA1, LA2]",41.300000,None,100.300000,0.0,"[{'lower': 135.0, 'display': '[135,153]', 'low...",Jiuquan SLC (Shuang Cheng Tzu)
23,"[ELA1, ELA2, Europa, ELA3, Diamant]",5.200000,None,-52.800000,0.0,"[{'lower': -10.5, 'display': '[-10.5,93.5]', '...",Guiana Space Center (Kourou)
28,"[LC81, LC1, LC175/59, LC1/LC31?, LC45P/2, LC20...",45.600000,None,63.400000,90.0,"[{'lower': 35.1, 'display': '[35.1,35.1]', 'lo...",Baikonur Cosmodrome (Tyuratam)
29,"[LC32/2?, LC16-2, LC132/1, LC32/2, LC43-4?, LC...",62.800000,None,40.600000,0.0,"[{'lower': -13.0, 'display': '[-13,62]', 'lowe...",Plesetsk Cosmodrome


## Launch Vehicles

In [11]:
operatorName = 'European Space Agency'
opCond = f"eq(launches.objects.operators.name,'{operatorName}')"

filter = f"{opCond}"
dataRequest = requestAllPages(pageName="/api/launch-vehicles",filter=filter,include='launches')

vehicleIndex = [d['id'] for d in dataRequest]
vehicleAttributes = [d['attributes'] for d in dataRequest]
vehicle_df = pd.DataFrame(vehicleAttributes,index=vehicleIndex)b

vehicleLaunches = []
for d in dataRequest:
    launchData = d['relationships']['launches']['data']
    vehicleLaunches.append([launch['id'] for launch in launchData])
vehicle_df['LaunchIDs'] = vehicleLaunches

vehicle_df.head(3)

,successfulLaunches,leoCapacity,height,span,diameter,failedLaunches,depth,mass,width,name,ssoCapacity,thrustLevel,escCapacity,geoCapacity,gtoCapacity,numStages,LaunchIDs
6,1,4900.0,None,None,4.0,0,None,240000.0,None,Ariane 40 H10+,NaN,752000.0,NaN,NaN,2050.0,3,[331]
7,97,23000.0,None,None,7.4,3,None,705000.0,None,Proton-M/Briz-M,NaN,1670000.0,NaN,3700.0,6150.0,4,"[6193, 6717, 5012, 5144, 4816, 5693, 5721, 634..."
25,9,4500.0,None,None,3.8,2,None,210000.0,None,Ariane 1,NaN,69300.0,730.0,NaN,1835.0,3,"[361, 806, 1357, 1405, 1452, 1745, 2654, 3216,..."


# Uberlog and ARES

In [7]:
resp_json = requestUberlog('gaia')
resp_json

{'count': 1691,
 'entries': [{'author': 'admin',
   'creationDate': 1518427647133,
   'eventDate': 1518427646000,
   'group': '',
   'id': '5a815dffd242d32bffd2da21',
   'mission': 'gaia',
   'severity': 'Error',
   'text': '<div>Next 2 passes not covered.</div>'}],
 'limit': 10,
 'skip': 0}

In [30]:
mission = 'gaia'
parameter = 'x1234'

t1 = time.strptime("01 Oct 2018 00:00", "%d %b %Y %H:%M")
t2 = time.strptime("02 Oct 2018 10:00", "%d %b %Y %H:%M")
intervalStart = int(time.mktime(t1)*1000.)
intervalEnd = int(time.mktime(t2)*1000.)

resp_json = requestAres(mission, parameter, intervalStart, intervalEnd, pageName="data", operation=None)
resp_json

[{'data': [{'timestamp': 1538300000000, 'value': 2.656369948440532},
   {'timestamp': 1538350000000, 'value': 1.0482780028455796},
   {'timestamp': 1538400000000, 'value': 0.5311632146975165}],
  'mission': 'gaia',
  'parameter': 'x1234'}]

# Request All DISCOS Data

## Objects

In [66]:
dataObj, _ = requestAllAndWait(pageName="/api/objects",sort='id',include='operators,states')

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [26]:
objectIndex = [int(d['id']) for d in dataObj]
objectAttributes = [d['attributes'] for d in dataObj]
object_df = pd.DataFrame(objectAttributes,index=objectIndex)
object_df.sort_index(inplace=True)

object_df.head()

,cosparId,name,depth,xSectMin,diameter,span,xSectAvg,satno,shape,objectClass,mass,height,width,vimpelId,xSectMax
1,1957-001A,Sputnik (8K71PS) Blok-A,NaN,5.309292,2.60,28.00,59.831632,1.0,Cyl,Rocket Body,3964.32,28.00,NaN,NaN,72.993346
2,1957-001B,Sputnik 1,0.58,0.264200,NaN,0.58,0.264200,2.0,Sphere,Payload,82.85,0.58,0.58,NaN,0.264200
3,1957-002A,Sputnik 2,NaN,0.785398,1.00,2.00,1.963495,3.0,Cone + Cyl,Payload,503.77,2.00,NaN,NaN,2.148686
4,1958-001A,Explorer 1,NaN,0.017671,0.15,2.03,0.247989,4.0,Cyl,Payload,13.88,2.03,NaN,NaN,0.305012
5,1958-002B,Vanguard 1,NaN,0.020106,0.16,0.16,0.020106,5.0,Sphere,Payload,1.46,NaN,NaN,NaN,0.020106


In [68]:
rel_att = ['operators', 'states']
object_df = data2df(dataObj, rel_att)

object_df

,cosparId,name,depth,xSectMin,diameter,span,xSectAvg,satno,shape,objectClass,mass,height,width,vimpelId,xSectMax,operators,states
1,1957-001A,Sputnik (8K71PS) Blok-A,NaN,5.309292,2.60,28.00,59.831632,1.0,Cyl,Rocket Body,3964.32,28.00,NaN,NaN,72.993346,[718],[264]
2,1957-001B,Sputnik 1,0.58,0.264200,NaN,0.58,0.264200,2.0,Sphere,Payload,82.85,0.58,0.58,NaN,0.264200,[672],[264]
3,1957-002A,Sputnik 2,NaN,0.785398,1.00,2.00,1.963495,3.0,Cone + Cyl,Payload,503.77,2.00,NaN,NaN,2.148686,[672],[264]
4,1958-001A,Explorer 1,NaN,0.017671,0.15,2.03,0.247989,4.0,Cyl,Payload,13.88,2.03,NaN,NaN,0.305012,"[321, 572]",[259]
5,1958-002B,Vanguard 1,NaN,0.020106,0.16,0.16,0.020106,5.0,Sphere,Payload,1.46,NaN,NaN,NaN,0.020106,[653],[259]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72953,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,Unknown,NaN,NaN,NaN,134006.0,NaN,None,None
72954,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,Unknown,NaN,NaN,NaN,76024.0,NaN,None,None
72955,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,Unknown,NaN,NaN,NaN,120702.0,NaN,None,None
72958,2022-085A,None,NaN,NaN,NaN,NaN,NaN,53239.0,None,Unknown,NaN,NaN,NaN,NaN,NaN,None,None


In [69]:
object_df.to_csv('all_discos_obj.csv')

In [70]:
len(object_df)

61928

## Fragmentations

In [21]:
dataFrag, incFrag = requestAllAndWait(pageName="/api/fragmentations",include="objects")

In [22]:
dataFrag[0]

{'type': 'fragmentation',
 'attributes': {'eventType': 'Unknown',
  'longitude': None,
  'comment': 'Event data is for epoch of 3 January 1980, given the uncertainty in actual time of separation event(s). It is likely that four of he 24 anomalous debris are associated with other space missions.',
  'altitude': None,
  'epoch': '1979-12-15',
  'latitude': None},
 'id': '53',
 'relationships': {'objects': {'links': {'self': '/api/fragmentations/53/relationships/objects',
    'related': '/api/fragmentations/53/objects'},
   'data': [{'type': 'object', 'id': '746'}]}},
 'links': {'self': '/api/fragmentations/53'}}

In [25]:
fragIndex = [int(d['id']) for d in dataFrag]
fragAttributes = [d['attributes'] for d in dataFrag]
frag_df = pd.DataFrame(fragAttributes,index=fragIndex)
frag_df['object'] = [ [int(data['id']) for data in d['relationships']['objects']['data']] for d in dataFrag]
frag_df.sort_index(inplace=True)

frag_df.head()

,eventType,longitude,comment,altitude,epoch,latitude,object
1,Zenit-2 Upper Stage,None,The Zenit second stage low thrust engine used ...,NaN,1985-12-28,None,[16436]
2,Proton Ullage Motor,331.900000,Parent satellite was one of two small engine u...,4730.0,1991-12-29,25.300000,[16443]
3,Ariane Upper Stage,42.000000,Event occurred approximately nine months after...,805.0,1986-11-13,7.000000,[16612]
4,Cosmos 699 Class (EORSAT),174.000000,Cosmos 1769 was the fifteenth spacecraft of th...,320.0,1987-09-21,-60.000000,[16892]
5,Deliberate,166.000000,USA 19 deliberately collided with USA 19 R/B a...,220.0,1986-09-05,15.000000,"[16934, 16935]"


In [26]:
frag_df.to_csv('all_discos_frag.csv')

In [27]:
len(frag_df)

650

## Launches

In [31]:
dataLaunch, incLaunch = requestAllAndWait(pageName="/api/launches",include="entities,objects,site,vehicle")

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

In [44]:
len(dataLaunch)

6257

In [40]:
rel_att = ['entities', 'objects', 'site', 'vehicle']

launchIndex = [int(d['id']) for d in dataLaunch]
launchAttributes = [d['attributes'] for d in dataLaunch]
launch_df = pd.DataFrame(launchAttributes,index=launchIndex)

for att in rel_att:
    launch_df[att] = [ getRelDataID(data['relationships'][att]['data']) for data in dataLaunch]
launch_df.sort_index(inplace=True)

launch_df.head()

,cosparLaunchNo,flightNo,epoch,failure,entities,objects,site,vehicle
1,1961-001,70D,1961-01-31T20:21:19+00:00,False,[259],"[58271, 70]",19.0,38.0
2,1961-002,L1-7,1961-02-04T01:18:03+00:00,False,[264],"[72, 71, 73]",28.0,91836.0
3,1962-071,S118,1962-12-19T01:25:45+00:00,False,[259],"[509, 514, 519, 523, 7256]",19.0,170.0
4,1963-001,T103-09,1963-01-04T08:49:00+00:00,False,[264],"[521, 524, 522]",28.0,91836.0
5,1963-004,358/D16,1963-02-14T05:35:00+00:00,False,[259],"[532, 553]",10.0,50.0


In [45]:
launch_df.to_csv('all_discos_launches.csv')

## Launch Sites

In [48]:
dataSite, _ = requestAllAndWait(pageName="/api/launch-sites",include="operators")

In [92]:
dataSite[0]

{'type': 'launchSite',
 'attributes': {'longitude': '-75.460000',
  'constraints': None,
  'altitude': 0.0,
  'name': 'Mid-Atlantic Regional Spaceport',
  'azimuths': None,
  'latitude': '37.850000',
  'pads': ['LP-0A', 'LP-0B']},
 'id': '2',
 'relationships': {'launches': {'links': {'self': '/api/launch-sites/2/relationships/launches',
    'related': '/api/launch-sites/2/launches'}},
  'operators': {'links': {'self': '/api/launch-sites/2/relationships/operators',
    'related': '/api/launch-sites/2/operators'},
   'data': [{'type': 'country', 'id': '259'}]}},
 'links': {'self': '/api/launch-sites/2'}}

In [49]:
len(dataSite)

45

In [52]:
rel_att = ['operators']

siteIndex = [int(d['id']) for d in dataSite]
siteAttributes = [d['attributes'] for d in dataSite]
site_df = pd.DataFrame(siteAttributes,index=siteIndex)

for att in rel_att:
    site_df[att] = [ getRelDataID(data['relationships'][att]['data']) for data in dataSite]
site_df.sort_index(inplace=True)

site_df.head()

,longitude,constraints,altitude,name,azimuths,latitude,pads,operators
1,None,None,NaN,Naro Space Center,None,None,None,[133]
2,-75.460000,None,0.0,Mid-Atlantic Regional Spaceport,None,37.850000,"[LP-0A, LP-0B]",[259]
3,46.300000,None,0.0,Kapustin Yar MSC,"[{'upperInc': True, 'lower': 72.0, 'lowerInc':...",48.600000,"[LC86/4, Mayak-2, LC86/1, LC107]",[203]
4,111.500000,None,0.0,Taiyuan SLC (Wuzhai),"[{'upperInc': True, 'lower': 350.0, 'lowerInc'...",37.800000,None,[50]
5,-3.100000,None,0.0,Hammaguira,"[{'upperInc': True, 'lower': 63.0, 'lowerInc':...",30.900000,[Brigitte],[84]


In [53]:
site_df.to_csv('all_discos_launch-sites.csv')

## Launch Vehicles

In [57]:
dataVehicle, _ = requestAllAndWait(pageName="/api/launch-vehicles",include="engines,family,stages")

In [58]:
len(dataVehicle)

358

In [63]:
rel_att = ['engines', 'family', 'stages']
vehicle_df = data2df(dataVehicle, rel_att)

vehicle_df.head()

,name,geoCapacity,leoCapacity,depth,ssoCapacity,span,height,escCapacity,gtoCapacity,numStages,successfulLaunches,failedLaunches,diameter,thrustLevel,width,mass,engines,family,stages
1,Naro-1,NaN,100.0,None,NaN,NaN,NaN,NaN,NaN,2.0,1,2,3.0,1670000.0,NaN,140000.0,"[1593, 1594]",1,"[787, 788]"
2,H-IIB,NaN,16500.0,None,NaN,NaN,NaN,NaN,8000.0,3.0,9,0,11.2,2256000.0,NaN,533400.0,"[854, 855, 856]",2,"[749, 350, 349]"
3,Proton-M/DM-3,3200.0,20000.0,None,NaN,NaN,NaN,NaN,NaN,4.0,3,2,7.4,1670000.0,NaN,691400.0,"[839, 1152, 1155, 1303]",3,"[316, 317, 318, 786]"
5,Long March (CZ) 11,NaN,700.0,None,350.0,NaN,NaN,NaN,NaN,4.0,13,0,2.0,NaN,NaN,58000.0,"[1620, 1621, 1572, 1573]",5,"[830, 831, 401, 416]"
6,Ariane 40 H10+,NaN,4900.0,None,NaN,NaN,NaN,NaN,2050.0,3.0,1,0,4.0,752000.0,NaN,240000.0,"[1494, 1446, 965]",6,"[369, 370, 372]"


In [64]:
vehicle_df.to_csv('all_discos_launch-vehicles.csv')

## Vehicle Families

In [70]:
dataFamily, _ = requestAllAndWait(pageName="/api/launch-vehicles/families", include="system")

In [71]:
dataFamily[0]

{'type': 'vehicleFamily',
 'id': '109',
 'attributes': {'name': 'SS-5'},
 'relationships': {'system': {'links': {'self': '/api/launch-vehicles/families/109/relationships/system',
    'related': '/api/launch-vehicles/families/109/system'},
   'data': {'type': 'launchSystem', 'id': '50'}},
  'vehicles': {'links': {'self': '/api/launch-vehicles/families/109/relationships/vehicles',
    'related': '/api/launch-vehicles/families/109/vehicles'}}},
 'links': {'self': '/api/launch-vehicles/families/109'}}

In [69]:
len(dataFamily)

123

In [74]:
rel_att = ['system']
family_df = data2df(dataFamily, rel_att)

family_df.head()

,name,system
1,KSLV-1 (Naro),1
2,H-II,2
3,Proton-M,3
4,SLV-P,4
5,Long March 11,5


In [75]:
family_df.to_csv('all_discos_launch-vehicle-families.csv')

## Launch Systems

In [78]:
dataSystem, _ = requestAllAndWait(pageName="/api/launch-systems",include="entities")

In [79]:
dataSystem[0]

{'type': 'launchSystem',
 'id': '2',
 'relationships': {'families': {'links': {'self': '/api/launch-systems/2/relationships/families',
    'related': '/api/launch-systems/2/families'}},
  'entities': {'links': {'self': '/api/launch-systems/2/relationships/entities',
    'related': '/api/launch-systems/2/entities'},
   'data': [{'type': 'country', 'id': '125'}]}},
 'attributes': {'name': 'H-Vehicle'},
 'links': {'self': '/api/launch-systems/2'}}

In [80]:
len(dataSystem)

62

In [81]:
rel_att = ['entities']
system_df = data2df(dataSystem, rel_att)

system_df.head()

,name,entities
1,KSLV,[133]
2,H-Vehicle,[125]
3,Proton (UR-500),[203]
4,SLV Series,[116]
5,Dong Feng,[50]


In [82]:
system_df.to_csv('all_discos_launch-systems.csv')

## Vehicle Engines

In [83]:
dataEngine, _ = requestAllAndWait(pageName="/api/launch-vehicles/engines")

In [84]:
dataEngine[0]

{'type': 'engine',
 'attributes': {'span': None,
  'thrustLevel': None,
  'name': 'SS-520',
  'width': None,
  'depth': None,
  'mass': 111.0,
  'height': 1.7,
  'maxIsp': None,
  'diameter': 0.52},
 'id': '81568',
 'relationships': {'vehicles': {'links': {'self': '/api/launch-vehicles/engines/81568/relationships/vehicles',
    'related': '/api/launch-vehicles/engines/81568/vehicles'}}},
 'links': {'self': '/api/launch-vehicles/engines/81568'}}

In [85]:
len(dataEngine)

847

In [90]:
rel_att = []
engine_df = data2df(dataEngine, rel_att)

engine_df.head()

,span,thrustLevel,name,width,depth,mass,height,maxIsp,diameter
836,NaN,509600.0,YF-77,NaN,None,2700.0,4.20,310.2,1.45
837,NaN,88260.0,YF-75D,NaN,None,550.0,2.80,442.0,1.50
838,NaN,1922000.0,RD-181,NaN,None,2200.0,3.68,311.9,2.21
839,NaN,1670000.0,RD-276,NaN,None,1280.0,2.72,288.0,1.50
840,NaN,1753800.0,AJ26-62,NaN,None,1235.0,3.70,331.0,2.00


In [91]:
engine_df.to_csv('all_discos_launch-vehicle-engines.csv')

## Vehicle Stages

In [95]:
dataStage, _ = requestAllAndWait(pageName="/api/launch-vehicles/stages", include="propellant")

In [96]:
dataStage[0]

{'type': 'stage',
 'attributes': {'wetMass': 147000.0,
  'span': None,
  'oxidiserMass': None,
  'fuelMass': None,
  'name': 'CZ-NGLV-300',
  'solidPropellantMass': None,
  'width': None,
  'depth': None,
  'burnTime': 160.0,
  'height': 26.3,
  'dryMass': 12000.0,
  'diameter': 3.35},
 'relationships': {'propellant': {'links': {'self': '/api/launch-vehicles/stages/311/relationships/propellant',
    'related': '/api/launch-vehicles/stages/311/propellant'},
   'data': {'type': 'propellant', 'id': '1'}},
  'vehicles': {'links': {'self': '/api/launch-vehicles/stages/311/relationships/vehicles',
    'related': '/api/launch-vehicles/stages/311/vehicles'}}},
 'id': '311',
 'links': {'self': '/api/launch-vehicles/stages/311'}}

In [97]:
len(dataStage)

677

In [99]:
rel_att = ['propellant']
stage_df = data2df(dataStage, rel_att)

stage_df.head()

,wetMass,span,oxidiserMass,fuelMass,name,solidPropellantMass,width,depth,burnTime,height,dryMass,diameter,propellant
311,147000.0,NaN,NaN,NaN,CZ-NGLV-300,NaN,NaN,NaN,160.0,26.3,12000.0,3.35,1.0
312,175600.0,NaN,NaN,NaN,CZ-5-500,158500.0,NaN,NaN,480.0,33.2,18000.0,5.00,2.0
313,26300.0,10.6,NaN,NaN,CZ-5-HO,22900.0,NaN,NaN,700.0,10.6,3400.0,5.00,2.0
314,12000.0,4.0,NaN,NaN,YZ-2,NaN,NaN,NaN,1105.0,4.0,5000.0,3.80,3.0
315,262600.0,NaN,177260.0,64740.0,Antares-200 first stage (RD-181),NaN,NaN,NaN,215.0,27.6,20600.0,3.90,4.0


In [100]:
stage_df.to_csv('all_discos_launch-vehicle-stages.csv')

## Propellants

In [102]:
dataProp, _ = requestAllAndWait(pageName="/api/propellants")

In [103]:
dataProp[0]

{'type': 'propellant',
 'relationships': {'stages': {'links': {'self': '/api/propellants/1/relationships/stages',
    'related': '/api/propellants/1/stages'}}},
 'id': '1',
 'attributes': {'oxidiser': 'LOX',
  'solidPropellant': None,
  'fuel': 'Kerosene'},
 'links': {'self': '/api/propellants/1'}}

In [104]:
len(dataProp)

42

In [105]:
rel_att = []
prop_df = data2df(dataProp, rel_att)

prop_df.head()

,oxidiser,solidPropellant,fuel
1,LOX,None,Kerosene
2,LOX,None,LH2 (Liquid Hydrogen)
3,N2O4,None,UDMH (Unsymmetrical Dimethyl Hydrazine)
4,LOX,None,RJ-1
5,None,TP-H8299,None


In [106]:
prop_df.to_csv('all_discos_propellants.csv')

## Reentries

In [112]:
dataReentry, _ = requestAllAndWait(pageName="/api/reentries",include="objects")

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

...

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [113]:
dataReentry[0]

{'type': 'reentry',
 'id': '29338',
 'attributes': {'epoch': '2022-03-21'},
 'relationships': {'objects': {'links': {'self': '/api/reentries/29338/relationships/objects',
    'related': '/api/reentries/29338/objects'},
   'data': [{'type': 'object', 'id': '16108'}]}},
 'links': {'self': '/api/reentries/29338'}}

In [114]:
len(dataReentry)

27122

In [116]:
rel_att = ['objects']
reentry_df = data2df(dataReentry, rel_att)

reentry_df.head()

,epoch,objects
1,1957-12-01,[1]
2,1958-01-03,[2]
3,1958-04-14,[3]
4,1970-03-31,[4]
5,1958-06-28,[6]


In [117]:
reentry_df.to_csv('all_discos_reentries.csv')

## Entities

In [6]:
dataEntity, _ = requestAllAndWait(pageName="/api/entities",include="launchSystems,hostCountry,launchSites")

In [7]:
dataEntity[0]

{'type': 'organisation',
 'relationships': {'launchSystems': {'links': {'self': '/api/entities/1229/relationships/launch-systems',
    'related': '/api/entities/1229/launch-systems'},
   'data': []},
  'objects': {'links': {'self': '/api/entities/1229/relationships/objects',
    'related': '/api/entities/1229/objects'}},
  'launches': {'links': {'self': '/api/entities/1229/relationships/launches',
    'related': '/api/entities/1229/launches'}},
  'hostCountry': {'links': {'self': '/api/entities/1229/relationships/host-country',
    'related': '/api/entities/1229/host-country'},
   'data': None},
  'launchSites': {'links': {'self': '/api/entities/1229/relationships/launch-sites',
    'related': '/api/entities/1229/launch-sites'},
   'data': []}},
 'id': '1229',
 'attributes': {'name': 'European Union',
  'dateRange': {'upperInc': False,
   'lower': '1952-07-23',
   'lowerInc': True,
   'upper': None,
   'empty': False,
   'display': '[1952-07-23,)'}},
 'links': {'self': '/api/entities/1

In [8]:
len(dataEntity)

1745

In [26]:
rel_att = ['launchSystems', 'hostCountry', 'launchSites']

ind = [int(d['id']) for d in dataEntity]
attributes = [d['attributes'] for d in dataEntity]
entity_df = pd.DataFrame(attributes,index=ind)

entity_df['type'] = [d['type'] for d in dataEntity]

for att in rel_att:
    if att=='hostCountry':
        entity_df[att] = [ getRelDataID(data['relationships'][att]['data']) if data['type']=='organisation' else None for data in dataEntity]
    else:
        entity_df[att] = [ getRelDataID(data['relationships'][att]['data']) for data in dataEntity]
entity_df.sort_index(inplace=True)

entity_df

,name,dateRange,type,launchSystems,hostCountry,launchSites
1,Afghanistan,NaN,country,None,NaN,None
2,Åland Islands,NaN,country,None,NaN,None
3,Albania,NaN,country,None,NaN,None
4,Algeria,NaN,country,None,NaN,[5]
5,American Samoa,NaN,country,None,NaN,None
...,...,...,...,...,...,...
2455,"ACME AtronOmatic, LLC","{'upperInc': False, 'lower': '2006-01-01', 'lo...",organisation,None,259.0,None
2456,Astrix Astronautics,"{'upperInc': False, 'lower': '2021-02-01', 'lo...",organisation,None,177.0,None
2457,Aurora Propulsion Technologies,"{'upperInc': False, 'lower': '2018-01-01', 'lo...",organisation,None,83.0,None
2458,E-Space,"{'upperInc': False, 'lower': '2020-01-01', 'lo...",organisation,None,259.0,None


In [27]:
entity_df.launchSystems.describe()

count       15
unique      15
top       [37]
freq         1
Name: launchSystems, dtype: object

In [28]:
entity_df.to_csv('all_discos_entities.csv')

## Initial Orbits

In [24]:
dataInitOrbit, _ = requestAllAndWait(pageName="/api/initial-orbits",sort='id',include="object")

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

...

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

In [31]:
data_same = []
for d in dataInitOrbit:
    if d['id']=='5':
        data_same.append(d)
        
pprint(data_same)

[{'attributes': {'aPer': 39.0,
                 'ecc': 0.008,
                 'epoch': '1965-05-30',
                 'frame': 'J2000',
                 'inc': 51.82,
                 'mAno': None,
                 'raan': None,
                 'sma': 6621000.0},
  'id': '5',
  'links': {'self': '/api/initial-orbits/5'},
  'relationships': {'object': {'data': {'id': '1383', 'type': 'object'},
                               'links': {'related': '/api/initial-orbits/5/object',
                                         'self': '/api/initial-orbits/5/relationships/object'}}},
  'type': 'initialOrbit'}]


In [28]:
len(dataInitOrbit)

44809

In [29]:
rel_att = ['object']
init_orbit_df = data2df(dataInitOrbit, rel_att)

init_orbit_df.head()

,inc,raan,mAno,aPer,frame,ecc,sma,epoch,object
1,63.44,NaN,NaN,64.0,J2000,0.011,6608000.0,1965-09-14,1451
2,70.18,NaN,NaN,85.0,J2000,0.003,6870000.0,1965-07-22,1447
3,31.74,NaN,NaN,254.0,J2000,0.016,7002000.0,1965-06-07,1385
4,51.81,NaN,NaN,40.0,J2000,0.011,6651000.0,1965-05-27,1382
5,51.82,NaN,NaN,39.0,J2000,0.008,6621000.0,1965-05-30,1383


In [9]:
rel_att = ['object']
init_orbit_df = data2df(dataInitOrbit, rel_att)

init_orbit_df.head()

,inc,raan,mAno,aPer,frame,ecc,sma,epoch,object
1,63.44,NaN,NaN,64.0,J2000,0.011,6608000.0,1965-09-14,1451
2,70.18,NaN,NaN,85.0,J2000,0.003,6870000.0,1965-07-22,1447
2,70.18,NaN,NaN,85.0,J2000,0.003,6870000.0,1965-07-22,1447
3,31.74,NaN,NaN,254.0,J2000,0.016,7002000.0,1965-06-07,1385
3,31.74,NaN,NaN,254.0,J2000,0.016,7002000.0,1965-06-07,1385


In [30]:
init_orbit_df.to_csv('all_discos_initial-orbits.csv')

## Destination Orbits

In [32]:
dataDestOrbit, _ = requestAllAndWait(pageName="/api/destination-orbits",sort='id',include="object")

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [33]:
dataDestOrbit[0]

{'type': 'destinationOrbit',
 'id': '36231',
 'attributes': {'inc': 52.0,
  'raan': 0.0,
  'mAno': None,
  'aPer': 0.0,
  'frame': 'J2000',
  'ecc': 0.00011,
  'sma': 7791700.0,
  'epoch': '1999-10-22'},
 'relationships': {'object': {'links': {'self': '/api/destination-orbits/36231/relationships/object',
    'related': '/api/destination-orbits/36231/object'},
   'data': {'type': 'object', 'id': '25941'}}},
 'links': {'self': '/api/destination-orbits/36231'}}

In [34]:
len(dataDestOrbit)

9847

In [35]:
rel_att = ['object']
dest_orbit_df = data2df(dataDestOrbit, rel_att)

dest_orbit_df.head()

,inc,raan,mAno,aPer,frame,ecc,sma,epoch,object
36231,52.0,0.0,NaN,0.0,J2000,0.00011,7791700.0,1999-10-22,25941
36232,52.0,0.0,NaN,0.0,J2000,0.00305,7299200.0,1999-11-29,25960
36233,98.1,0.0,NaN,0.0,J2000,0.00124,7029600.0,1999-12-10,25975
36234,45.0,0.0,NaN,0.0,J2000,0.00062,7200300.0,1999-12-09,25977
36235,45.0,0.0,NaN,0.0,J2000,0.00062,7200200.0,1999-12-09,25982


In [40]:
dest_orbit_df.to_csv('all_discos_destination-orbits.csv')